# Humanos vs Caballos

En los anteriores ejercicios hemos podido ver a las redes neuronales convolucionales en acción y cómo su naturaleza a través de los kernels mejora el rendimiento drásticamente.

Sin embargo, el dataset de Fashion MNIST es un dataset bastante irreal en términos de espacio y de tamaño para tareas de visión artificial reales. Una de las limitaciones más grandes al entrenar redes convolucionales es que no siempre se puede cargar todo el dataset a la memoria RAM. La memoria RAM es un recurso limitado en cualquier servidor, por lo que se debe buscar una forma más eficiente de poder realizar este tipo de tareas.

En este notebook exploraremos el caso con un dataset más cercano a lo que un ingeniero se puede encontrar *en la vida real* y cómo tratar con el mismo,


## El dataset: Humanos vs Caballos
Usaremos un dataset preparado por Laurence Moroney de Google el cual contiene imágenes sintéticas de caballos y personas. Primero tenemos que descargar tanto el conjunto de entrenamiento como el conjunto de validación o pruebas.

In [1]:
!wget --no-check-certificate \
    https://storage.googleapis.com/download.tensorflow.org/data/horse-or-human.zip \
    -O data/horse-or-human.zip

--2024-06-05 18:09:27--  https://storage.googleapis.com/download.tensorflow.org/data/horse-or-human.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 2800:3f0:4003:c01::cf, 2800:3f0:4003:c02::cf, 2800:3f0:4003:c08::cf, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|2800:3f0:4003:c01::cf|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 149574867 (143M) [application/zip]
Saving to: ‘data/horse-or-human.zip’

data/horse-or-human 100%[===================>] 142.65M  6.57MB/s    in 22s     

2024-06-05 18:09:50 (6.50 MB/s) - ‘data/horse-or-human.zip’ saved [149574867/149574867]



In [2]:
!wget --no-check-certificate \
    https://storage.googleapis.com/download.tensorflow.org/data/validation-horse-or-human.zip \
    -O data/validation-horse-or-human.zip

--2024-06-05 18:09:52--  https://storage.googleapis.com/download.tensorflow.org/data/validation-horse-or-human.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 2800:3f0:4003:c02::cf, 2800:3f0:4003:c08::cf, 2800:3f0:4003:c00::cf, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|2800:3f0:4003:c02::cf|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 11480187 (11M) [application/zip]
Saving to: ‘data/validation-horse-or-human.zip’

data/validation-hor 100%[===================>]  10.95M  5.66MB/s    in 1.9s    

2024-06-05 18:09:55 (5.66 MB/s) - ‘data/validation-horse-or-human.zip’ saved [11480187/11480187]



En la siguiente celda usaremos funciones de la librería os, para acceder a librerías del sistema operativo de tal manera que podamos acceder al sistema de archivos de la máquina.


In [9]:
import os
import zipfile

local_zip = 'data/horse-or-human.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('data/horse-or-human')
local_zip = 'data/validation-horse-or-human.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('data/validation-horse-or-human')
zip_ref.close()

Los contenidos de los archivos comprimidos .zip se extraerán en el directorio base /tmp/horse-or-human, dentro del mismo se tendrán 2 subdirectorios: `horse` y `human`

El **conjunto de entrenamiento** corresponde a los datos que se usan para decirle a la red neuronal 'así se ve un caballo'o 'así se ve una persona'.

Algo que considerar es que no estamos etiquetando explícitamente las imágenes como caballos o personas. Esto se logrará usando un dataset especial de Pytorch que es capaz de usar la estructura de nuestro dataset.

In [34]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder
from torchvision.transforms import ToTensor, Normalize, Compose

train_dataset = ImageFolder(root="data/horse-or-human", transform=ToTensor())
test_dataset = ImageFolder(root="data/validation-horse-or-human", transform=ToTensor())

train_dataloader = DataLoader(train_dataset, batch_size=16)
test_dataloader = DataLoader(test_dataset, batch_size=16)

In [47]:
print(len(train_dataset), len(test_dataset))
print(train_dataset[0][0].shape)

1027 256
torch.Size([3, 300, 300])


## Definiendo el modelo


In [36]:
# dispositivo de entrenamiento
device = (
    "cuda" if torch.cuda.is_available()
    else "mps" if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Usando: {device}")

Usando: mps


In [37]:
class ConvNet(nn.Module):
    def __init__(self):
        super().__init__()
        # self.flatten = nn.Flatten()
        self.backbone = nn.Sequential(
            nn.Conv2d(in_channels=3, out_channels=32, kernel_size=3),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(in_channels=128, out_channels=128, kernel_size=3),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
        )
        self.head = nn.Sequential(
            nn.Linear(in_features=32768, out_features=128),
            nn.ReLU(),
            nn.Linear(in_features=128, out_features=2),
        )

    def forward(self, x):
        features = self.backbone(x)
        # flatten tensor
        flat_features = features.view(features.size(0), -1)
        # apply the classifier
        logits = self.head(flat_features)
        return logits
    
model = ConvNet().to(device)
print(model)

ConvNet(
  (backbone): Sequential(
    (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1))
    (4): ReLU()
    (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1))
    (7): ReLU()
    (8): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (9): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1))
    (10): ReLU()
    (11): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (head): Sequential(
    (0): Linear(in_features=32768, out_features=128, bias=True)
    (1): ReLU()
    (2): Linear(in_features=128, out_features=2, bias=True)
  )
)


Agregamos las capas convolucionales similar al ejemplo anterior, y *aplanamos* el resultado final para continuar con las capas densamente conectadas.

Seguidamente añadimos las capas densamente conectadas.

Nótese que debido a que usamos un problema de clasificación binaria, la salida de nuestra red neuronal será una función [*sigmoide*](https://wikipedia.org/wiki/Sigmoid_function), de tal manera que la salida ser'a un escalar entre 0 y 1, se puede interpretar este valor como una probabilidad de que la imagen pertenezca a la clase 1.


In [29]:
# bucle de entrenamiento

def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    # modo entrenamiento
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        # mover si es necesario
        X = X.to(device)
        y = y.to(device)
        # forward
        # prediccion y costo
        pred = model(X)
        loss = loss_fn(pred, y)

        # backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * batch_size + len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

# funcion de pruebas
def test_loop(dataloader, model, loss_fn):
    # modo evaluacion
    model.eval()
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0
    # evaluacion del modelo con torch.no_grad
    with torch.no_grad():
        for X, y in dataloader:
            X = X.to(device)
            y = y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

### Entrenamiento
Se entrenará la red por 15 épocas. Esto puede tomar algunos minutos en completarse.

Monitoree los valores en cada época.

In [38]:
learning_rate = 1e-3
batch_size = 16
epochs = 5
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.RMSprop(model.parameters(), lr=learning_rate)
# bucle principal
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer)
    test_loop(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 0.777415  [   16/ 1027]
Test Error: 
 Accuracy: 50.0%, Avg loss: 18.177686 

Epoch 2
-------------------------------
loss: 30.953892  [   16/ 1027]
Test Error: 
 Accuracy: 50.0%, Avg loss: 9.486739 

Epoch 3
-------------------------------
loss: 16.683067  [   16/ 1027]
Test Error: 
 Accuracy: 50.0%, Avg loss: 19.736915 

Epoch 4
-------------------------------
loss: 33.590099  [   16/ 1027]
Test Error: 
 Accuracy: 50.0%, Avg loss: 10.818101 

Epoch 5
-------------------------------
loss: 18.315487  [   16/ 1027]
Test Error: 
 Accuracy: 50.0%, Avg loss: 15.511246 

Done!


### Haciendo predicciones

Vamos a usar el modelo entrenado haciendo predicciones. La celda a continuación nos permitirá subir uno o más archivos para realizar predicciones sobre los mismos.

In [57]:
import numpy as np
import cv2
from torchvision.io import read_image
from torchvision import transforms
import matplotlib.pyplot as plt
from torchvision.transforms import Resize
# %matplotlib.inline

# Define the necessary transformations
preprocess = transforms.Compose([
    transforms.Resize((300, 300)),
    # transforms.CenterCrop(224),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# img = cv2.cvtColor(cv2.imread("caballo2.jpg"), cv2.COLOR_BGR2RGB)
# img = cv2.resize(img, (150, 150))
img = read_image("caballo.jpg").float() / 255.0

print(img.shape)
# plt.imshow(img)
# plt.show()
x = preprocess(img)
x = x.unsqueeze(0)
x = x.to(device)
# x = Resize((150, 15))(x)
print(x.shape)
model.eval()
with torch.no_grad():
    pred = model(x)
print(f"prediction: {pred}, {pred.argmax(1).item()}")
# print(pred.ar)
# if classes[0]>0.5:
    # print(" is a human")
# else:
    # print(" is a horse")
 

torch.Size([3, 400, 800])
torch.Size([1, 3, 300, 300])
prediction: tensor([[-34.9537,  18.8797]], device='mps:0'), 1


### Visualizando Representaciones Internas

Tambien es divertido visualizar qué tipo de filtros y qué tipo de características nuestra red ha aprendido a resaltar.

Seleccionando una imagen aleatoria en el conjunto de entrenamiento, se puede generar una figura con la salida de cada capa convolucional representando a cada filtro y su correspondiente mapa de características, las mismas se suelen llamar **representaciones internas** de la red neuronal.
